In [42]:
import time
import datetime
import csv
import pandas as pd 
import re
import os
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait  # for implicit and explict waits
from selenium.webdriver.chrome.options import Options  # for suppressing the browser
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.webdriver.common.by import By

In [43]:
s=Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=s)



====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Driver [C:\Users\Oaki\.wdm\drivers\chromedriver\win32\100.0.4896.60\chromedriver.exe] found in cache


In [44]:
def scroll_to_bottom(driver):

    old_position = 0
    new_position = None

    while new_position != old_position:
        # Get old scroll position
        old_position = driver.execute_script(
                ("return (window.pageYOffset !== undefined) ?"
                 " window.pageYOffset : (document.documentElement ||"
                 " document.body.parentNode || document.body);"))
        # Sleep and Scroll
        time.sleep(1)
        driver.execute_script((
                "var scrollingElement = (document.scrollingElement ||"
                " document.body);scrollingElement.scrollTop ="
                " scrollingElement.scrollHeight;"))
        # Get new position
        new_position = driver.execute_script(
                ("return (window.pageYOffset !== undefined) ?"
                 " window.pageYOffset : (document.documentElement ||"
                 " document.body.parentNode || document.body);"))

In [ ]:
def parse_country_data(country_text):
  lines = country_text.split("\n")
  data = []
  for each_line in lines:
    try:
      result = re.search(">(.*?)</", each_line)
      if ">" in result.group(1):
        result = re.search(">(.*?)$", result.group(1))

      data.append(result.group(1))
    except:
      continue
  return data

In [45]:
def craw_week_data(soup):
    df_week = pd.DataFrame(columns=["#", "Date", "Country,Other", "Cases in the last 7 days", "Cases in the preceding 7 days", "Weekly case /%/ change", "Cases in the last 7 days/1M pop", "Deaths in the last 7 days", "Deaths in the preceding 7 days", "Weekly Death /%/ change", "Deaths in the last 7 days/1M pop", "Population", "Continent"])
    country_odd_data = soup.find_all("tr", attrs={"class":"odd"})
    country_even_data = soup.find_all("tr", attrs={"class":"even"})
    try:
        for i in range(len(country_odd_data)):
            date = datetime.datetime.today()
            result = parse_country_data(str(country_odd_data[i]))
            date = date.strftime('%d:%m:%Y')
            if result[0] != '':
                result.insert(1, date)
                df_length = len(df_week)
                df_week.loc[df_length] = result  
            result = parse_country_data(str(country_even_data[i]))
            if result[0] != '':
                result.insert(1, date)
                df_length = len(df_week)
                df_week.loc[df_length] = result  
    except:
        pass
    return df_week

In [46]:
driver.get("https://www.worldometers.info/coronavirus/weekly-trends/")
time.sleep(1)
scroll_to_bottom(driver)


In [47]:
html = driver.page_source
soup_week = BeautifulSoup(html, 'html.parser')
df_week = craw_week_data(soup_week)
df_week

,#,Date,"Country,Other",Cases in the last 7 days,Cases in the preceding 7 days,Weekly case /%/ change,Cases in the last 7 days/1M pop,Deaths in the last 7 days,Deaths in the preceding 7 days,Weekly Death /%/ change,Deaths in the last 7 days/1M pop,Population,Continent


In [49]:
html

'<html lang="en"><!--<![endif]--><head><script type="text/javascript" async="" src="https://static.criteo.net/js/ld/publishertag.prebid.113.js"></script><script async="" src="//c.amazon-adsystem.com/aax2/apstag.js"></script><script type="text/javascript" src="https://m.addthis.com/live/red_lojson/300lo.json?si=625e8e92da41ba00&amp;bkl=0&amp;bl=1&amp;pdt=711&amp;sid=625e8e92da41ba00&amp;pub=ra-54615fa823b2af68&amp;rev=v8.28.8-wp&amp;ln=en&amp;pc=men&amp;cb=0&amp;ab=-&amp;dp=www.worldometers.info&amp;fp=coronavirus%2Fweekly-trends%2F&amp;fr=&amp;of=0&amp;pd=0&amp;irt=0&amp;vcl=0&amp;md=0&amp;ct=1&amp;tct=0&amp;abt=0&amp;cdn=0&amp;pi=1&amp;rb=0&amp;gen=100&amp;chr=UTF-8&amp;colc=1650364052097&amp;jsl=4097&amp;uvs=625e8e922dadf1e0000&amp;skipb=1&amp;callback=addthis.cbs.jsonp__094640917872534260"></script><script type="text/javascript" src="https://v1.addthisedge.com/live/boost/ra-54615fa823b2af68/_ate.track.config_resp"></script><script src="//js-sec.indexww.com/ht/p/184310-82987131453484

In [48]:
if not os.path.isfile('Corona_week.tsv'):
    df_week.to_csv("Corona_week.tsv", sep = "\t", mode = "a", index = False)
else:
    df_week.to_csv("Corona_week.tsv", sep = "\t", mode = "a", index = False, header = False)